In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data_path = '/kaggle/input/monthly-milk-production-pounds/monthlyMilkProduction.csv'
df = pd.read_csv(data_path)
df.head()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader

# Set random seed for reproducibility
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the dataset
data_path = '/kaggle/input/monthly-milk-production-pounds/monthlyMilkProduction.csv'
df = pd.read_csv(data_path)

# Assuming the dataset has a column 'Monthly milk production: pounds per cow' for the time series
time_series = df['Monthly milk production: pounds per cow. Jan 62 ? Dec 75'].values.astype(float)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
time_series_normalized = scaler.fit_transform(time_series.reshape(-1, 1))

# Create sequences for LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 12  # Using 12 months (1 year) as the sequence length
X, y = create_sequences(time_series_normalized, seq_length)

# Split into training and testing sets (80% train, 20% test)
train_size = int(len(X) * 0.8)
X_train = X[:train_size]
y_train = y[:train_size]
X_test = X[train_size:]
y_test = y[train_size:]

# Convert to PyTorch tensors
X_train = torch.FloatTensor(X_train).to(device)
y_train = torch.FloatTensor(y_train).to(device)
X_test = torch.FloatTensor(X_test).to(device)
y_test = torch.FloatTensor(y_test).to(device)

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Instantiate the model
model = LSTMModel(input_size=1, hidden_size=50, num_layers=1).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training the model
num_epochs = 250
train_losses = []

for epoch in range(num_epochs):
    model.train()
    outputs = model(X_train)
    optimizer.zero_grad()
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    train_losses.append(loss.item())
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    train_predictions = model(X_train).cpu().numpy()
    test_predictions = model(X_test).cpu().numpy()

# Inverse transform the predictions and actual values
train_predictions = scaler.inverse_transform(train_predictions)
y_train_actual = scaler.inverse_transform(y_train.cpu().numpy())
test_predictions = scaler.inverse_transform(test_predictions)
y_test_actual = scaler.inverse_transform(y_test.cpu().numpy())

# Plot 1: Training Loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig('training_loss.png')
plt.show()
plt.close()

# Plot 2: Actual vs Predicted (Training Set)
plt.figure(figsize=(15, 5))
plt.plot(y_train_actual, label='Actual Milk Production (Train)', color='blue')
plt.plot(train_predictions, label='Predicted Milk Production (Train)', color='orange', linestyle='--')
plt.title('Actual vs Predicted Milk Production (Training Set)')
plt.xlabel('Time Step')
plt.ylabel('Milk Production (Pounds)')
plt.legend()
plt.grid(True)
plt.savefig('train_predictions.png')
plt.show()
plt.close()

# Plot 3: Actual vs Predicted (Test Set)
plt.figure(figsize=(15, 5))
plt.plot(y_test_actual, label='Actual Milk Production (Test)', color='blue')
plt.plot(test_predictions, label='Predicted Milk Production (Test)', color='orange', linestyle='--')
plt.title('Actual vs Predicted Milk Production (Test Set)')
plt.xlabel('Time Step')
plt.ylabel('Milk Production (Pounds)')
plt.legend()
plt.grid(True)
plt.savefig('test_predictions.png')
plt.show()
plt.close()

# Plot 4: Full Time Series with Predictions
full_actual = time_series[seq_length:train_size + seq_length]
full_pred = np.concatenate([train_predictions, test_predictions])
plt.figure(figsize=(20, 5))
plt.plot(time_series, label='Actual Milk Production (Full)', color='blue')
plt.plot(range(seq_length, seq_length + len(full_pred)), full_pred, label='Predicted Milk Production', color='orange', linestyle='--')
plt.title('Full Time Series with Predictions')
plt.xlabel('Time Step')
plt.ylabel('Milk Production (Pounds)')
plt.legend()
plt.grid(True)
plt.savefig('full_time_series.png')
plt.show()
plt.close()

print("Plots saved as 'training_loss.png', 'train_predictions.png', 'test_predictions.png', and 'full_time_series.png'")